## Router

Given a query, picks one of several query engines to execute a query. For example it can direct you towards a summarizer or a question/answer engine.

In [2]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [3]:
# Allows Jupyter to play nice with async
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Vector Index vs Summary Index

An index is a set of metadata over our data and different indexes will have different retrieval behaviors.

### Vector Index

Indexes nodes via text embeddings. Core abstraction in any RAG system. Querying a vector index returns the most similar node by embedding similarity.

### Summary Index

Querying returns all the nodes currently in the index.

In [7]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [8]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [9]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Selectors

* LLM Selectors use the LLM to output a JSON that is parsed and the corresponding indexes are queried.
* The Pydantic selectors use the OpenAI Function Calling API to produce pydantic selection objects, rather than parsing raw JSON

In [10]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [11]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through role specialization, workflow management, and efficient communication mechanisms. It utilizes executable feedback to improve code generation quality and achieves state-of-the-art performance in software development tasks. The document also details the development process of a "Drawing App" using MetaGPT, highlighting stages like requirement gathering, UI design, system architecture, testing, and task allocation. It discusses the role of different agents, the use of Python libraries for GUI design, and the importance of structured messaging for effective communication. Additionally, it addresses the performance of MetaGPT in generating executable code, limitations, and ethical concerns related to its capabilities and user in

In [12]:
print(len(response.source_nodes))

34


In [14]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context, which would be necessary to understand how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages and subscribe to relevant messages based on their profiles. This shared message pool allows all agents to exchange messages directly, enabling them to access messages from other entities transparently. By storing information in this global message pool, agents can retrieve required information without the need to inquire about other agents and wait for their responses, thereby enhancing communication efficiency.


In [15]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [16]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results demonstrate the effectiveness of MetaGPT in addressing challenges related to information overload and reducing hallucinations in software generation. By utilizing a global message pool and a subscription mechanism, MetaGPT successfully streamlines communication and filters out irrelevant contexts, enhancing the relevance and utility of information. This design is crucial in ensuring efficiency and accuracy in software development tasks.
